In [ ]:
# pip install PyPDF2
# pip install fpdf

######## ALTERNATIVE with annoying watermark
# pip install aspose-words

In [1]:
import os
import glob
import json
import ast
# import md2pdf

from openai import OpenAI

def load_api_key(config_file):
    with open(config_file) as f:
        config = json.load(f)
    client = OpenAI(api_key=config['api_key'])
    return client

def get_text_files(directory):
    list_of_files = glob.glob(os.path.join(directory, '*.txt'))
    latest_file, file_name = [""," "]
    if len(list_of_files) > 0:
        latest_file = max(list_of_files, key=os.path.getctime)
        full_name = os.path.basename(latest_file)
        file_name = os.path.splitext(full_name)
    return latest_file, file_name[0]

def read_file(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    return content

def output_directories(path,name,extension):
    if not os.path.exists(path):
        os.makedirs(path)
    n_files = ""
    # n_files = len(get_text_files(path))
    # n_files = n_files + 1
    # if n_files < 10:
    #     n_files = "0" + str(n_files)
    # else:
    #     n_files = str(n_files)
    nameFile = name + n_files + extension
    return(path + nameFile)

# from __future__ import print_function

def find_values(id, json_repr):
    results = []

    def _decode_dict(a_dict):
        try:
            results.append(a_dict[id])
        except KeyError:
            pass
        return a_dict

    json.loads(json_repr, object_hook=_decode_dict) # Return value ignored.
    return results

directory = "./mealPlans/"
config_file = "config.json"

client = load_api_key(config_file)


In [33]:

def get_completion(prompt, client, model="gpt-4", temperature=0, ):

    messages = [{"role": "system", "content": 'You are a useful and concise inventory manager in a grocery store'},
                {"role": "user", "content": prompt}]
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    # response = completion.model_dump_json(indent=2)
    return completion.choices[0].message.content # message["content"]



def get_completion_from_messages(messages, client, model="gpt-4", temperature=0):

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]

In [3]:
###################################
####### Ingredients  ############
###################################
# INPUT: Lists of Ingredients for each day in Danish.
# OUTPUT: English translation of the lists of Ingredients for each day.

# Setup new file of MealPlans

text_file, file_name = get_text_files(directory)

output_Ingredients_dir = "./outputIngredients/"
outputIngredientsFile = output_directories(output_Ingredients_dir, file_name + "_Ingredients_",".txt")

content = read_file(text_file)
prompt = f"""
Your task is to translate the  
lists of ingredients from Danish to English.\

Lists of ingredients: 
"""

content_prompt = f"""
```{content}```
"""
response = get_completion(prompt+content_prompt, client)

print(response)

#######################################
#######################################
# Saving results

f = open(outputIngredientsFile, "w")
f.write(response)
f.close()  
    

Here are the translations:

Oven-baked falafels

INGREDIENTS

2 cans chickpeas in brine
1 onion
3 cloves garlic
1 handful parsley
2 eggs
2 tsp cumin
2 tsp chili sauce (optional)
2 tbsp lemon juice
1 tbsp oil
2 tsp salt
1 dl breadcrumbs
Dressing
2.5 dl sour cream
2 cloves garlic
1 tsp mustard
salt and pepper
optional chopped chives or parsley
Sides
4 pita bread or tortillas
300 g salad
2 tomatoes
0.5 cucumber

Spicy chicken meatballs with chickpea salad and bread

INGREDIENTS

Chicken meatballs
400 g minced chicken max 10% fat
1 egg
2 tbsp oatmeal
1 regular onion
1 clove garlic
1 tsp cumin
1 tsp paprika
optional chili
1 tsp salt
pepper
Chickpea salad
200 g chickpeas, dried or 2 cans chickpeas
250 g cherry tomatoes
0.5 cucumber
1 bell pepper
200 g peas
1 regular onion, finely chopped
optional 1 bunch chopped parsley if you have
Dressing for salad
4 tbsp lemon juice
2 tbsp oil
2 cloves garlic
0.5 tsp chili
plenty of salt and pepper
Sides
1 pack whole grain baguettes

Tart with tomato, oni

In [32]:
###################################
####### ShoppingList  ############
###################################
# INPUT: Lists of Ingredients for each day in English.
# OUTPUT: Json object containing a table with the ingredients 
#   grouped by category and the quantities added up.

# Setup new file of ShoppingLists

text_file, file_name = get_text_files(output_Ingredients_dir)

output_ShoppingList_dir = "./outputShoppingList/"
outputShoppingListFile_txt = output_directories(output_ShoppingList_dir, file_name + "_ShoppingList_text",".txt")

content = read_file(text_file)
prompt = f"""

From the list of ingredients between <>, perform the following tasks: \
1. isolate the ingredients. \
2. group the ingredients by type and add up the quantities. \
3. make a shopping list grouping the type of ingredient using the following categories: \
- produce. \
- meat. \
- canned goods. \
- eggs, milk, yogurt, sour cream. \
- cheese. \
- cold cuts. \
- nuts. \
- spices. \
- 

If there are no ingredients in one category, do not list the category in the shopping list.

Output only the shopping list.

Lists of ingredients: 
"""

content_prompt = f"""
```{content}```
"""
response = get_completion(prompt+content_prompt, client)

print(response)

#######################################
#######################################
# Saving results

f = open(outputShoppingListFile_txt, "w")
f.write(response)
f.close()  
    

**Ingredients by Type and Quantities:**

**Vegetables:**
- Onion: 8
- Garlic: 16 cloves
- Parsley: 2 handfuls
- Salad: 300g
- Tomatoes: 2
- Cucumber: 2
- Cherry Tomatoes: 500g
- Bell Pepper: 2
- Peas: 200g
- Iceberg Lettuce: 1
- Corn: 1 can
- Carrots: 6
- Zucchini: 1
- Cauliflower: 1
- Leeks: 2
- Beetroots: 600g
- Potatoes: 1kg

**Proteins:**
- Chickpeas: 4 cans
- Minced Chicken: 400g
- Salad Cheese: 200g
- Cod Roe: 1 can
- Red Lentils: 250g
- Chicken Thighs: 1kg

**Dairy:**
- Eggs: 7
- Sour Cream: 7.5 dl
- Low-fat Mozzarella: 150g
- Skimmed Milk: 10 dl
- Cooking Cream: 250 ml

**Grains:**
- Breadcrumbs: 1 dl
- Pita Bread/Tortillas: 4
- Oatmeal: 104g
- Whole Grain Baguettes: 3 packs
- Graham Flour: 150g
- Wheat Flour: 6 tbsp
- Whole Grain Rolls: 4
- Whole Grain Lasagna Sheets: 12

**Spices & Condiments:**
- Cumin: 4 tsp
- Chili Sauce: 2 tsp
- Lemon Juice: 7 tbsp
- Salt: 7 tsp
- Pepper: 7 tsp
- Paprika: 2 tsp
- Chili: 1.5 tsp
- Oil: 5 tbsp
- Rapeseed Oil: 2 tbsp
- Mustard: 1 tsp
- Mayon

In [30]:
from fpdf import FPDF
# from PyPDF2 import PdfMerger 

def create_pdf(input_file):
    # Create a new FPDF object
    pdf = FPDF()

    # Open the text file and read its contents
    with open(input_file, 'r') as f:
        text = f.read()

    # Add a new page to the PDF
    pdf.add_page()

    # Set the font and font size
    pdf.set_font('Arial', size=12)

    # Write the text to the PDF
    pdf.write(5, text)

    # Save the PDF
    pdf.output('output.pdf')

    # # If a template PDF is specified, merge it with the new PDF
    # merger = PdfMerger()
    # template_pdf = './template.pdf'
    # if template_pdf:
    #     merger.append(PdfMerger(open(template_pdf, 'rb')))
    #     merger.append(PdfMerger(open('output.pdf', 'rb')))
    #     merger.write('merged_output.pdf')

In [31]:


create_pdf(outputShoppingListFile_txt)

FileNotFoundError: [Errno 2] No such file or directory: './template.pdf'

In [4]:
###################################
####### ShoppingList  ############
###################################
# INPUT: Lists of Ingredients for each day in English.
# OUTPUT: Json object containing a table with the ingredients 
#   grouped by category and the quantities added up.

# Setup new file of ShoppingLists

text_file, file_name = get_text_files(output_Ingredients_dir)

output_ShoppingList_dir = "./outputShoppingList/"
outputShoppingListFile_json = output_directories(output_ShoppingList_dir, file_name + "_ShoppingList_",".json")

content = read_file(text_file)
prompt = f"""
Your task is to create a table with all the ingredients grouped
by category and quantity.\

The output table must be in Json format.\

Your response must only be a Json object and nothing else.\

Lists of ingredients: 
"""

content_prompt = f"""
```{content}```
"""
response = get_completion(prompt+content_prompt, client)

print(response)

#######################################
#######################################
# Saving results

f = open(outputShoppingListFile_json, "w")
f.write(response)
f.close()  
    

{
  "Oven-baked falafels": {
    "Vegetables": {
      "onion": 1,
      "garlic": 3,
      "parsley": 1,
      "tomatoes": 2,
      "cucumber": 0.5
    },
    "Canned goods": {
      "chickpeas in brine": 2
    },
    "Dairy": {
      "eggs": 2,
      "sour cream": 2.5
    },
    "Spices and condiments": {
      "cumin": 2,
      "chili sauce": 2,
      "lemon juice": 2,
      "oil": 1,
      "salt": 2,
      "mustard": 1,
      "pepper": 1
    },
    "Bakery": {
      "breadcrumbs": 1,
      "pita bread or tortillas": 4
    },
    "Produce": {
      "salad": 300
    }
  },
  "Spicy chicken meatballs with chickpea salad and bread": {
    "Meat": {
      "minced chicken max 10% fat": 400
    },
    "Vegetables": {
      "onion": 2,
      "garlic": 3,
      "cucumber": 0.5,
      "bell pepper": 1,
      "cherry tomatoes": 250,
      "peas": 200
    },
    "Canned goods": {
      "chickpeas": 2
    },
    "Dairy": {
      "egg": 1
    },
    "Spices and condiments": {
      "cumin": 2,
 

In [18]:
###################################
####### ShoppingList  ############
###################################
# INPUT: Lists of Ingredients for each day in English.
# OUTPUT: Json object containing a table with the ingredients 
#   grouped by category and the quantities added up.

# Setup new file of ShoppingLists

text_file, file_name = get_text_files(output_Ingredients_dir)

output_ShoppingList_dir = "./outputShoppingList/"
outputShoppingListFile_md = output_directories(output_ShoppingList_dir, file_name + "_ShoppingList_markdown",".md")

content = read_file(text_file)
prompt = f"""
Create a shopping list with all the ingredients grouped
by aisle and all the ingredients quantities added up together.\

Lists of ingredients: 
"""

content_prompt = f"""
```{content}```
"""
response = get_completion(prompt+content_prompt, client)

print(response)

#######################################
#######################################
# Saving results

f = open(outputShoppingListFile_md, "w")
f.write(response)
f.close()  
    

Here is your shopping list grouped by aisle:

**Produce Aisle:**
- 2 onions
- 3 cloves garlic
- 1 handful parsley
- 300 g salad
- 2 tomatoes
- 0.5 cucumber
- 1 regular onion
- 250 g cherry tomatoes
- 0.5 cucumber
- 1 bell pepper
- 200 g peas
- 1 regular onion, finely chopped
- 250 g cherry tomatoes
- 2 regular onions
- 0.5 iceberg lettuce
- 0.5 cucumber
- 2 carrots
- 1 large onion
- 1 large carrot
- 1 zucchini
- 2 leeks
- 600 g beetroots
- 500 g potatoes
- 2 onions
- 3 cloves garlic
- 1 tbsp fresh ginger

**Canned Goods Aisle:**
- 4 cans chickpeas in brine
- 2 cans chopped tomatoes
- 1 can corn
- 1 can cod roe 200 g

**Bakery Aisle:**
- 4 pita bread or tortillas
- 1 pack whole grain baguettes
- 2 whole grain baguettes
- 4 large whole grain rolls

**Dairy Aisle:**
- 2 eggs
- 2.5 dl sour cream
- 1 egg
- 200 g low-fat salad cheese in cubes 3%
- 5 eggs
- 1 dl sour cream 5%
- 2 tbsp low-fat sour cream 5%
- 75 g low-fat mozzarella 13%
- 75 g low-fat mozzarella 13%
- 5 dl skimmed milk
- 250 m

In [17]:
# This code example demonstrates how to convert a Markdoen file to a PDF document.
import aspose.words as aw

# Load an existing Markdown document
doc = aw.Document(outputShoppingListFile)

# Save the document as PDF
doc.save(file_name + "_ShoppingList_markdown.pdf")

In [ ]:
# import json
# outputShoppingListFile = './outputShoppingList/w52_plan_Ingredients_03_ShoppingList_03.json'
# json_Shopping = json.loads(outputShoppingListFile)


In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt

# df = pd.read_json(outputShoppingListFile)
# df.head()

,Vegetables,Proteins,Grains,Dairy,Canned Goods,Spices and Condiments,Oils,Others
Onions,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bell Peppers,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Garlic Cloves,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Spring Onions,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Carrots,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
